Copyright (C) 2023 Pablo Castells y Alejandro Bellogín

El código que contiene este notebook se ha implementado para la realización de las prácticas de la asignatura "Búsqueda y minería de información" de 4º del Grado en Ingeniería Informática, impartido en la Escuela Politécnica Superior de la Universidad Autónoma de Madrid. El fin del mismo, así como su uso, se ciñe a las actividades docentes de dicha asignatura.

### **Búsqueda y Minería de Información 2022-23**
### Universidad Autónoma de Madrid, Escuela Politécnica Superior
### Grado en Ingeniería Informática, 4º curso

# Bloque 1 &ndash; Sistemas de recomendación

Fechas:

* Comienzo: martes 28 / jueves 30 de marzo
* Entrega: lunes 8 de mayo, 23:59

## Objetivos

Este primer bloque de la práctica tiene por objetivo la implementación y evaluación eficiente de sistemas de recomendación. En este bloque se desarrollarán:

* Estructuras para el manejo de datos de interacción entre usuarios e items ("ratings" para simplificar).
* Algoritmos de recomendación basada en filtrado colaborativo.
* Métricas de evaluación de sistemas de recomendación.

## Material proporcionado

Se proporcionan software y datos para la realización de la práctica:

* Un esqueleto de clases y funciones donde el estudiante desarrollará sus implementaciones. 
  - De modo similar a las prácticas anteriores, se proporciona una celda de prueba al final de este notebook que deberá funcionar con las implementaciones del estudiante.
  - Junto a la celda de prueba en este mismo notebook, se muestra como referencia un ejemplo de salida generada con una implementación de los profesores.
* Los siguientes conjuntos de datos de ratings por usuarios a items:
  - Dos conjuntos de juguete para prueba y depuración: <ins>toy1.csv</ins> (se genera en Matrices.ipynb) y <ins>toy2.csv</ins> (proporcionado en el curso Moodle) con ratings ficticios.
  - Un conjunto de datos reales de ratings a películas: *ml-1m.zip* disponible en la Web de [MovieLens](https://grouplens.org/datasets/movielens/1m). De los archivos disponibles, se utilizará sólamente <ins>ratings.dat</ins>, añadiéndole una cabecera `u::i::r::t`.
  
Los esqueletos de código que se proporcionan aquí son a modo de guía: el estudiante puede modificarlo todo libremente, siempre que la celda de prueba funcione correctamente **sin cambios**.

## Calificación

El peso de esta práctica (recomendación + redes sociales) en la nota final de prácticas es del **40%**.

La calificación se basará en el **número** de ejercicios realizados y la **calidad** de los mismos. La puntuación que se indica en cada apartado es orientativa, en principio se aplicará tal cual se refleja pero podrá matizarse por criterios de buen sentido si se da el caso.

Para dar por válida la realización de un ejercicio, el código deberá funcionar (a la primera) integrado con las clases que se facilitan. El profesor comprobará este aspecto ejecutando la celda de prueba y otras adicionales.

La corrección de las implementaciones se observará por la **coherencia de los resultados** (por ejemplo, las métricas sobre los algoritmos de recomendación), y se valorará la eficiencia en tiempo de ejecución.

## Entrega

La entrega consistirá en dos ficheros tipo *notebook* (uno para recomendación y otro para redes sociales) donde se incluirán todas las **implementaciones** solicitadas en cada ejercicio, así como una explicación de cada uno a modo de **memoria**.

## Indicaciones

La realización de los ejercicios conducirá en muchos casos a la implementación de funciones y/o clases adicionales a las que se indican en el enunciado. Algunas vendrán dadas por su aparición en los propias celdas de prueba, y otras por conveniencia a criterio del estudiante.

Igual que en prácticas anteriores, no deberán editarse las celdas de prueba. Estas celdas deberán ejecutar sin errores a la primera con el código entregado por el estudiante (naturalmente con salvedad de los ejercicios que no se hayan implementado.

## Ejercicio 1: Estructuras de datos y recomendación simple (2pt)

#### 1.1 &nbsp; Estructuras de datos

Implementar las clases necesarias para manejar **datos de entrada y prueba** (ratings) para los algoritmos de recomendación. La funcionalidad se implementará en una clase Ratings, que permitirá leer los datos de un fichero de texto, así como un método que genere dos particiones aleatorias de entrenamiento y test, para evaluar y comparar la efectividad de diferentes algoritmos de recomendación.

In [ ]:
import numpy as np
import pandas as pd

class Ratings:
    def __init__(self, file=False, sep=','):
        # Código aquí...
        
    def matrix(self):
        # Código aquí...
    
    def nusers(self):
        # Código aquí...
    
    def nitems(self):
        # Código aquí...
    
    def nratings(self):
        # Código aquí...
        
    # uidx can be an int or an array-like of ints.
    def uidx_to_uid(self, uidx):
        # Código aquí...
        
    # iidx can be an int or an array-like of ints.
    def iidx_to_iid(self, iidx):
        # Código aquí...
    
    # uid can be an int or an array-like of ints.
    def uid_to_uidx(self, uid):
        # Código aquí...
        
    # iid can be an int or an array-like of ints.
    def iid_to_iidx(self, iid):
        # Código aquí...
        
    def iidx_rated_by(self, uidx):
        # Código aquí...
        
    def uidx_who_rated(self, iidx):
        # Código aquí...
        
    # You may use scikit learn split here if you wish.
    def random_split(self, ratio):
        # Código aquí...
    
    #
    # The remaining functions are just for debugging purposes
    #

    def rating(self, uid, iid):
        # Código aquí...

    def items_rated_by(self, uid):
        # Código aquí...
        
    def users_who_rated(self, iid):
        # Código aquí...
    
    def user_ratings(self, uid):
        # Código aquí...

    def item_ratings(self, iid):
        # Código aquí...
    
    # To inspect random data splits.
    def save(self, file):
        df = pd.DataFrame(columns=self.iids, index=self.uids, data=self.m).unstack().reset_index(name='r')
        df.columns = ['i', 'u', 'r']
        df = df[df.r>0][['u', 'i', 'r']].sort_values(by=['u', 'i'])
        df.to_csv(file, index=False)

#### 1.2 &nbsp;  Recomendaciones: métodos simples no personalizados

La **salida** de un recomendador consistirá en un diccionario con un ránking por usuario. 

Implementar un primer **recomendador simple** por rating promedio en una clase `AverageRecommender`. El recomendador sólo recomendará ítems que tengan un mínimo número de ratings, que se indicará como parámetro en el constructor (con ello se mejora el acierto de la recomendación). Se proporciona una clase `MajorityRecommender` a modo de ejemplo en el que el estudiante podrá basarse, así como `RandomRecommender`, que se utiliza en ocasiones como referencia en experimentos. 

**Importante**: recordar que no deben recomendarse los ítems que los usuarios ya hayan puntuado.

In [132]:
# Suggestion: compute the scores in the recommenders' constructor.

from itertools import islice

# Given a matrix, returns a matrix of positions of top k values per row.
def top_positions_per_row(m, k):
    return np.sort(np.argpartition(m, -k)[:, -k:], axis=1)[:, ::-1]

# Given a matrix and a set of indices per rows, returns the matrix values for the indices. 
# This function is used in the Recommender class and metrics classes.
def get_elements(m, indices, cutoff=np.inf):
    return np.array([s[t[0:min(cutoff, len(t))]] for s, t in zip(m, indices)])

class Recommendation:
    def __init__(self, scores, n, training):
        # Código aquí...

    def ranked_iidx(self):
        # Código aquí...
        
    def scores(self):
        # Código aquí...
    
    def recommendation(self, uid):
        # Código aquí...
        
    # Format the recommendation as a string for the first n users.
    def display(self, n):
        r = ''
        for uid in islice(self.recomendation, n):
            r += '    User ' + str(uid) + ' -> <' 
            for iid, score in self.recommendation(uid): r += str(iid) + ':' + str(score) + ' '
            r = (r[:-1] + '>\n') if len(self.recommendation(uid)) > 0 else r + 'empty>\n'
        return r[:-1]
        
class Recommender():
    def __init__(self, training):
        self.training = training

    def __repr__(self):
        return type(self).__name__

    def recommend(self, n):
        return Recommendation(self.scores, n, self.training)

class RandomRecommender(Recommender):
    def __init__(self, training):
        super().__init__(training)
        self.scores = np.random.random(training.matrix().shape)

class MajorityRecommender(Recommender):
    def __init__(self, training, threshold=0):
        super().__init__(training)
        # training.matrix() >= threshold creates a mask with 'True' on relevant ratings and 'False' everywhere
        # else. Thus 'pop' is an array with the counts of relevant ratings of each item.
        pop = np.sum(training.matrix() >= threshold, axis=0)
        # This product by a vector of ones (of user-row length) creates a matrix where the pop vector gets
        # copied on all rows; the recommendation is not personalized and ranking is the same for all users 
        # -- except of course in the end different training items will be filtered out for different users.
        self.scores = np.outer(np.ones(training.nusers()), pop)

class AverageRecommender(Recommender):
    def __init__(self, training, minr=0):
        # Código aquí...

### Ejercicio 1 &ndash; Explicación/documentación

(por hacer)

## Ejercicio 2: Filtrado colaborativo kNN (2pt)

Implementar un algoritmo de filtrado colaborativo mediante vecinos próximos orientado a usuarios por *similitud coseno* (sin normalizar por la suma de similitudes). 

In [125]:
class CosineUserSimilarity:
    def __init__(self, training):
        # Código aquí...

class UserKNNRecommender(Recommender):
    def __init__(self, training, sim, k):
        super().__init__(training)
        # Código aquí...

### Ejercicio 2 &ndash; Explicación/documentación

(por hacer)

## Ejercicio 3: Evaluación (1pt)

Se desarrollarán clases que permitan calcular métricas para evaluar y comparar el acierto de los recomendadores: se implementarán **precisión** y **recall**. 

Como resumen de este bloque, se incluirá una *tabla con los valores de las métricas* (dos columnas) más el tiempo de ejecución (una columna más) sobre todos los algoritmos implementados (filas), al menos para el conjunto de datos de <ins>MovieLens 1M</ins>. En el caso de ser capaces de procesar un conjunto de datos más grande, se documentará el tamaño en RAM de la matriz de ratings.

Opcionalmente, se podrán implementar otras métricas a elección del estudiante (nDCG, etc.), cuya prueba se incluirá en la función `student_test_recsys()` del ejercicio 4 ("ampliaciones").

In [128]:
class Metric():
    def __init__(self, test, cutoff):
        self.test = test
        self.cutoff = cutoff

    def __repr__(self):
        return type(self).__name__ + ('@' + str(self.cutoff) if self.cutoff != np.inf else '')

class Precision(Metric):
    def __init__(self, test, cutoff=np.inf, threshold=1):
        super().__init__(test, cutoff)
        # Código aquí...

    def compute(self, recommendation):
        # Código aquí...
        
class Recall(Metric):
    def __init__(self, test, cutoff=np.inf, threshold=1):
        super().__init__(test, cutoff)
        # Código aquí...

    def compute(self, recommendation):
        # Código aquí...

### Ejercicio 3 &ndash; Explicación/documentación

(por hacer)

Ejemplo de tabla de resumen:

||Precision@K|Recall@K|Tiempo de ejecución
|-|:-:|:-:|:-:
|Algoritmo 1|...|...|...
|Algoritmo 2|...|...|...
|...|...|...|...
|Algoritmo n|...|...|...

## Ejercicio 4: Ampliaciones (1pt)

Elegir uno de los siguientes ejercicios:

* Implementar dos variantes de kNN a elección del estudiante, por ejemplo: kNN normalizado, vecinos próximos orientado a ítem, similitud de Pearson, kNN centrado en la media. Indicación: para kNN normalizado el algoritmo exigirá un mínimo de ratings de vecinos para aceptar recomendar un ítem (con ello se mejora el acierto de la recomendación, de forma similar a la recomendación por rating promedio).
* Implementar filtrado colaborativo mediante factorización de matrices.
* Crear una implementación de las estructuras de ratings con matrices dispersas, de forma que sea posible generar recomendaciones sobre conjuntos de datos más grandes, tales como [MovieLens 10M](https://grouplens.org/datasets/movielens/10m) y [MovieLens 25M](https://grouplens.org/datasets/movielens/25m).

Para probar las implementaciones deberá completarse la función `student_test_recsys()` para ilustrar la ejecución de las variantes adicionales, y se incluirán las filas que correspondan en la tabla del apartado anterior.

In [127]:
# Código aquí: clases, funciones...

def student_test_recsys():
    # Código de prueba aquí...

### Ejercicio 4 &ndash; Explicación/documentación

(por hacer)

## Celda de prueba

Descarga los ficheros de datos y coloca sus contenidos en una carpeta **recsys-data** en el mismo directorio que este *notebook*.

In [135]:
import datetime
import time

# Test data structures and algorithms on a dataset.
def test(ratings_file, example_user, example_item, k, minr, topn=np.inf, cutoff=np.inf, threshold=1, sep=','):
    print('Reading the data at', time.strftime('%X...'))
    start = time.time()
    ratings = Ratings(ratings_file, sep)
    print(f'Ratings matrix takes {round(10 * ratings.matrix().nbytes / 1024 / 1024) / 10:,} MB in RAM')
    timer(start)

    # Test Ratings class on the dataset.
    test_data(ratings, example_user, example_item)
    
    # Produce a rating split and test a set of recommenders. 
    train, test = ratings.random_split(0.8)
    metrics = [Precision(test, cutoff=cutoff, threshold=threshold), 
               Recall(test, cutoff=cutoff, threshold=threshold)]
    run_recommenders(train, metrics, k, minr, topn)

# Test the rating data handling code (Ratings class).
def test_data(ratings, example_user, example_item):
    print('-------------------------\nTesting the ratings data structures')
    print(f'{ratings.nratings():,} ratings by {ratings.nusers():,} users on {ratings.nitems():,} items')
    print('Ratings of user', example_user, ':', ratings.user_ratings(example_user))
    print('Ratings of item', example_item, ':', ratings.item_ratings(example_item))

# Run and evaluate some recommenders on rating data.
def run_recommenders(train, metrics, k, minr, topn):
    print('-------------------------')
    start = time.time()
    run_recommender(RandomRecommender(train), metrics, topn)
    timer(start)
    
    print('-------------------------')
    start = time.time()
    run_recommender(MajorityRecommender(train, threshold=4), metrics, topn)
    timer(start)
    
    print('-------------------------')
    start = time.time()
    run_recommender(AverageRecommender(train, minr), metrics, topn)
    timer(start)
    
    print('-------------------------')
    start = time.time()
    print('Creating user cosine similarity')
    sim = CosineUserSimilarity(train)
    timer(start)
    start = time.time()
    print('Creating kNN recommender')
    knn = UserKNNRecommender(train, sim, k)
    timer(start)
    start = time.time()
    run_recommender(knn, metrics, topn)
    timer(start)
    
# Run a recommender and evaluate a list of metrics on its output.
def run_recommender(recommender, metrics, topn):
    print('Testing', recommender, '(top', str(topn) + ')')
    recommendation = recommender.recommend(topn)
    print('Four example recommendations:\n' + recommendation.display(4))
    for metric in metrics:
        print(metric, '=', metric.compute(recommendation))

def timer(start):
    print('--> elapsed time:', datetime.timedelta(seconds=round(time.time() - start)), '<--')
    
np.random.seed(0) # This should make most executions equivalent and comparable (e.g as to the random split and the random recommender).
print('=========================\nTesting toy 1 dataset')
test('recsys-data/toy1.csv', example_user='v', example_item='b', k=4, min=2, topn=4, cutoff=4)
print('=========================\nTesting toy 2 dataset')
test('recsys-data/toy2.csv', example_user=1, example_item=2, k=4, min=2, topn=4, cutoff=4)
print('=========================\nTesting MovieLens \'1 million\' dataset')
test('recsys-data/ratings-1m.dat', example_user=200, example_item=1000, k=10, minr=3, topn=10, cutoff=10, threshold=4, sep='::')
print('=========================\nDone.')

# Additional testing?
student_test_recsys()

Testing toy 1 dataset
Reading the data at 15:49:28...
Ratings matrix takes 0.0 MB in RAM
--> elapsed time: 0:00:00 <--
-------------------------
Testing the ratings data structures
11 ratings by 4 users on 5 items
Ratings of user v : {'b': 4.0, 'c': 5.0, 'd': 3.0}
Ratings of item b : {'v': 4.0, 'x': 2.0, 'y': 4.0}
-------------------------
Testing RandomRecommender (top 4)
Four example recommendations:
    User v -> <a:0.978618342232764 e:0.11827442586893322>
    User x -> <c:0.9446689170495839 d:0.5218483217500717>
    User y -> <d:0.5684339488686485 e:0.018789800436355142>
    User z -> <e:0.6818202991034834 a:0.6176354970758771 c:0.6169339968747569 b:0.6120957227224214>
Precision@4 = 0.0625
Recall@10 = 0.0
--> elapsed time: 0:00:00 <--
-------------------------
Testing MajorityRecommender (top 4)
Four example recommendations:
    User v -> <a:1.0 e:1.0>
    User x -> <c:2.0 d:1.0>
    User y -> <d:1.0 e:1.0>
    User z -> <b:2.0 c:2.0 a:1.0 e:1.0>
Precision@4 = 0.0625
Recall@10 = 0.

### Salida obtenida por el estudiante

*(por hacer)*